In [3]:
import io
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from timezonefinder import TimezoneFinder
import time
from glob import glob
import rasterio as rio
from googleapiclient.discovery import build
from google.oauth2 import service_account
from googleapiclient.http import MediaIoBaseDownload
import numpy as np
from osgeo import gdal
gdal.SetConfigOption('SHAPE_RESTORE_SHX', 'YES')
import ee
import geemap
import ast
import re


In [4]:
def apply_scale_and_offset(image):

    # Band aliases.
    BLUE = 'CMI_C01'
    RED = 'CMI_C02'
    VEGGIE = 'CMI_C03'
    GREEN = 'GREEN'
    
    # Number of bands in the EE asset, 0-based.
    NUM_BANDS = 33
    
    # Skipping the interleaved DQF bands.
    BLUE_BAND_INDEX = (1 - 1) * 2
    RED_BAND_INDEX = (2 - 1) * 2
    VEGGIE_BAND_INDEX = (3 - 1) * 2
    GREEN_BAND_INDEX = NUM_BANDS - 1
    
    # Visualization range for GOES RGB.
    GOES_MIN = 0.0
    GOES_MAX = 0.7  # Alternatively 1.0 or 1.3.
    GAMMA = 1.3
    bands = [None] * NUM_BANDS  # Initialize with None to ensure correct length.
    
    for i in range(1, 17):
        band_name = f'CMI_C{str(100 + i)[-2:]}'
        offset = ee.Number(image.get(f'{band_name}_offset'))
        scale = ee.Number(image.get(f'{band_name}_scale'))
        bands[(i - 1) * 2] = image.select(band_name).multiply(scale).add(offset)

        dqf_name = f'DQF_C{str(100 + i)[-2:]}'
        bands[(i - 1) * 2 + 1] = image.select(dqf_name)

    # Green = 0.45 * Red + 0.10 * NIR + 0.45 * Blue
    green1 = bands[RED_BAND_INDEX].multiply(0.45)
    green2 = bands[VEGGIE_BAND_INDEX].multiply(0.10)
    green3 = bands[BLUE_BAND_INDEX].multiply(0.45)
    green = green1.add(green2).add(green3)
    bands[GREEN_BAND_INDEX] = green.rename(GREEN)

    return ee.Image(ee.Image(bands).copyProperties(image, image.propertyNames()))

In [5]:
def fetch_gee_data(start_date, end_date, coords, fps):

    Map = geemap.Map()
    
    hotspot_date = start_date
    
    roi = ee.Geometry.Rectangle(coords)
    
    date_of_interest = ee.Date(hotspot_date)
    
    ## VIIRS TRUE COLOUR
    viirs = ee.ImageCollection("NASA/VIIRS/002/VNP09GA").filterDate(start_date, end_date).filterBounds(roi)

    # FC: L2 VIIRS I BAND, RED/INFRARED/SWIR
    rgb_viirs_tc = viirs.select(['M5', 'M4', 'M3'])
    
    obj = TimezoneFinder()
    timezone = obj.timezone_at(lng=float(coords[0]), lat=float(coords[1]))
    
    # Set the goes time to be 1:30pm to coincide with VIIRS overpass
    doi = ee.Date(hotspot_date + 'T13:30:00', timezone)
    
    # Subtract the time of each image in collection from date of interest
    rgb_viirs_tc_sort = rgb_viirs_tc.map(lambda image: image.set(
        'dateDist',
        ee.Number(image.get('system:time_start')).subtract(doi.millis()).abs()
    ))
    
    # sort in ascending order by dateDist (so top image will correspond to date of interest)
    viirs_ic_rc_sorted = rgb_viirs_tc_sort.sort('dateDist')
    
    # grab the first image from the sorted image collection
    img_viirs_tc = viirs_ic_rc_sorted.first()
    
    # clip the image to the roi
    clipped_viirs_tc = img_viirs_tc.clip(roi)
    
    rgb_vis_viirs_tc = {'min': 0.0, 'max': 0.3}

    ## LANDSAT TRUE COLOUR
    
    # Add more time to the landsat retrieval 
    landsat_start = (datetime.strptime(start_date,"%Y-%m-%d") - timedelta(days=16)).strftime("%Y-%m-%d")
    landsat_end = (datetime.strptime(end_date,"%Y-%m-%d") + timedelta(days=16)).strftime("%Y-%m-%d")
    
    landsat = ee.ImageCollection("LANDSAT/LC08/C02/T1_TOA").filterDate(landsat_start, landsat_end).filterBounds(roi)
    
    true_color_432 = landsat.select(['B4', 'B3', 'B2'])
    
    # Subtract the time of each image in collection from date of interest
    landsat_sort = true_color_432.map(lambda image: image.set(
        'dateDist',
        ee.Number(image.get('system:time_start')).subtract(doi.millis()).abs()
    ))
    
    # grab the first image from the sorted image collection
    landsat_tc = landsat_sort.first()
    
    # clip the image to the roi
    clipped_landsat = landsat_tc.clip(roi)
    
    true_color_432_vis = {'min': 0.0, 'max': 0.4}

    ## GOES TRUE COLOUR

    goes = ee.ImageCollection("NOAA/GOES/16/MCMIPF").filterDate(start_date, end_date).filterBounds(roi)

    # Set the goes time to be 1:30pm to coincide with VIIRS overpass
    goes_date = ee.Date(hotspot_date + 'T13:30:00', timezone)

    goes_sort = goes.map(lambda image: image.set(
            'dateDist',
            ee.Number(image.get('system:time_start')).subtract(goes_date.millis()).abs()))

    # sort in ascending order by dateDist (so top image will correspond to date of interest)
    goes_sorted = goes_sort.sort('dateDist')

    # grab the first image from the sorted image collection
    goes_img = goes_sorted.first()

    # clip the image to the roi
    #goes_img_clipped = goes_img.clip(roi)

    #goes_img_tc = apply_scale_and_offset(ee.Image(goes_img_clipped))
    goes_img_tc = apply_scale_and_offset(ee.Image(goes_img))

    # Band aliases.
    BLUE = 'CMI_C01'
    RED = 'CMI_C02'
    GREEN = 'GREEN'
    
    # Visualization range for GOES RGB.
    GOES_MIN = 0.0
    GOES_MAX = 0.7  # Alternatively 1.0 or 1.3.
    GAMMA = 1.3

    goes_rgb_viz = {'bands': [RED, GREEN, BLUE], 'min': GOES_MIN, 'max': GOES_MAX, 'gamma': GAMMA}

    Map.centerObject(roi, 11)
    Map.addLayer(goes_img_tc, goes_rgb_viz, 'goes', shown=0)
    Map.addLayer(clipped_landsat, true_color_432_vis, 'C-landsat', shown=0)
    Map.addLayer(landsat_tc, true_color_432_vis, 'F-landsat', shown=0)
    Map.addLayer(clipped_viirs_tc, rgb_vis_viirs_tc, 'C-viirs-tc', shown=1)
    Map.addLayer(img_viirs_tc, rgb_vis_viirs_tc, 'F-viirs-tc', shown=0)
    
    hotspots = ee.Geometry.MultiPoint(fps)
    Map.addLayer(hotspots, name='fp hotspots', shown=1)

    return Map

In [14]:
fp_file = pd.read_excel(r'C:\Users\kzammit\Documents\plumes\dfs\all-false-positives-2023-07-01.xlsx')

Maps = []
hotspots = []

for idx, row in fp_file.iterrows():

    start_date = row['start_date']
    start_date = str(start_date).split(' ')[0]

    end_date = row['end_date']
    end_date = str(end_date).split(' ')[0]
    
    coords = row['bounds']
    coords = coords.split('(')[1]
    coords = coords.split(')')[0]
    coords = ast.literal_eval("[" + coords + "]")
    
    fps = row['geometry']
    fps_format = re.findall(r"\(\s*(-?\d+\.\d+)\s+(-?\d+\.\d+)\s*\)", fps)
    fp_points = [[float(lon), float(lat)] for lon, lat in fps_format]
    
    Map = fetch_gee_data(start_date, end_date, coords, fp_points)

    Maps.append(Map)

In [15]:
Maps[1]

Map(center=[58.745134917756566, -97.42029999996957], controls=(WidgetControl(options=['position', 'transparent…